In [1]:
with open('H:/College WOrk/papers/stop-words.txt',encoding='utf-16') as fp:
    v = fp.read()

def removestopwords(line):
    #print("original line",line)
    querywords = line.split()

    resultwords  = [word for word in querywords if word.lower() not in v]
    result = ' '.join(resultwords)
    #print(result)
    return result

In [2]:
from nltk.tokenize import word_tokenize
def tokenizefortfidf(originalsen):
    tokenized_sents = [word_tokenize(i) for i in originalsen]
    sen_list1=[]
    for i in tokenized_sents:
        sen_list1=sen_list1+i
    return sen_list1
#print(sen_list1)

In [3]:
def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(list(zip(wordlist,wordfreq)))

In [4]:
def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

In [5]:
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 

In [6]:
def computeTF(wordDict, doc):
    tfDict = {}
    corpusCount = len(doc)
    for word, count in wordDict.items():
        tfDict[word] = count/float(corpusCount)
    return(tfDict)

In [7]:
def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    #print("this is doclist",docList)
    
#     idfDict = dict.fromkeys(docList.keys(),docList.values())
#     print(idfDict)
    for word, val in docList.items():
        #print(word,"word and value ",val)
        idfDict[word] = math.log10(N / float(val) + 1)
        
    return(idfDict)

In [8]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return(tfidf)

In [9]:
lst=[]
def loopsecond(word1len,word1,idf,j):
    #print("inside function")
    for k in range(word1len): 
            #print("inside for loop")
            #print("this is index",idf.index[0])
            #print("this is word",word1[0])
            if idf.index[j] == word1[k]:
                lst.append(idf['ratio'][j])
                #print(idf.index[j])
                #print(word1[k])
                #print("true")
                break
    #total = sum(lst)
    #print("list is",lst)

def loopfirst(leng,word1,idf):
    word1len = len(word1)
    for j in range(leng):
        #print(j)
        loopsecond(word1len,word1,idf,j)

In [10]:
def convert_list_to_string(org_list, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(org_list)

In [39]:
import codecs
import string
import re
from bs4 import BeautifulSoup  
from nltk.tokenize import word_tokenize
from indicnlp.tokenize import sentence_tokenize
from indicnlp.tokenize import indic_tokenize
import pandas as pd
import sklearn as sk
import math
import io

class ShortText:
    def __init__(self, my_id, human_label, summary, short_text):
        self.id = my_id         
        self.human_label = human_label    
        self.summary = summary 
        self.short_text = short_text
    def __str__(self):
        '''
        For printing purposes.
        '''
        return '%d\t%d\t%s\t%s' % (self.id, self.human_label, self.summary, self.short_text)

def load_file(filename,createfile):
    tokenize_words=[]
    originalsen=[]
    #file1 = open(createfile, "w")
    #retrieve the original text 
    with codecs.open(filename, encoding='utf-8') as f:
        data = f.read()
    #use beautifulsoup to get tag attributes and elements
    soup = BeautifulSoup(data)
    docno=soup.find_all('docno')
    
    text=soup.find_all('text')
    #store in a dictionary with ShortText Instances as values
    instances = {}
    my_id = 0
    for n,tit in zip(docno,text):
        #print("hello2")
        tit=tit.get_text()
        tit=str(tit)
        print("this is text",tit)
        #s = re.sub('[!#?,.:";]', '', tit)
        for tokenize in indic_tokenize.trivial_tokenize(tit):
            tokenize_words.append(tokenize)
        # Convert list of strings to string
        originalfull_str = convert_list_to_string(tokenize_words)
        senlateron=[]
        sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
        for t in sentences:
            senlateron.append(t)
        originalsen=[]
        sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
        for t in sentences:
            sen=removestopwords(t)
            originalsen.append(sen)
            #print(originalsen)
        sen_listfortfidf=tokenizefortfidf(originalsen)
        #print(sen_listfortfidf)
        list3 = [''.join(c for c in s if c not in string.punctuation) for s in sen_listfortfidf]
        list3 = [s for s in list3 if s]
        dictionary = wordListToFreqDict(list3)
        sorteddict = sortFreqDict(dictionary)
        context=listToString(list3)
        wordDictA = dict.fromkeys(list3, 0)
        for word in list3:
            wordDictA[word]+=1
        df2=pd.DataFrame([wordDictA])
        tfFirst = computeTF(wordDictA, context)
        tf = pd.DataFrame([tfFirst])
        idfs = computeIDF(wordDictA)
        idfFirst = computeTFIDF(tfFirst, idfs)
        #idfSecond = computeTFIDF(tfSecond, idfs)
        #putting it in a dataframe
        idf= pd.DataFrame([idfFirst])
        idf = idf.transpose()
        idf.columns=['ratio']
        final_df = pd.DataFrame(columns=['sentence', 'score'])

        for i in range(len(originalsen)):
            originalsen[i]=str(originalsen[i])[1:-1]
            word = listToString(originalsen[i])
   
            word1 = word.split()
    
            lenofdf=len(idf)
            loopfirst(lenofdf,word1,idf)
            total = sum(lst)
            final_df = final_df.append({'sentence': originalsen[i], 'score': total},ignore_index='True')
    
            lst.clear()
        final_df.sort_values(by=['score'], inplace=True, ascending=False)
        #final_df1=final_df.sort_values(by=['score'], inplace=True, ascending=False)
        #print(final_df)
        final_df2=final_df['sentence'].head(round(40*len(final_df.index)/100))
        #print("this is final_df2",final_df2)
        listtoretrieve=list(final_df2.index.values) 
        listtoretrieve.sort()
        #print(listtoretrieve)
#         #final_df.sort_values(by=['score'], inplace=True, ascending=False)
        #from original list retrive
        df2 = pd.DataFrame(senlateron,columns=['sentences'])
        new_list = df2.values.tolist()
        res1=[]

        for x in new_list:
            res1=res1+x
        finaldraft=""
        for i in listtoretrieve:
            finaldraft+=res1[i]
        finaldraft=" ".join(finaldraft.split())   
        #print(finaldraft)
        with io.open(createfile, "w", encoding="utf-8") as f:
            f.write(finaldraft)
       

    return instances

In [40]:
import os

#for root, dirs, files in os.walk("/home/harsh/Documents/gujdocs/gsf2003/"):
for root, dirs, files in os.walk("H:/gujarati Text summarization dataset/economics/"):   
#for root, dirs, files in os.walk("G:/Movies/post_1/"):
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(root, file), 'r') as f:
                #print(os.path.join(root, file))
                load_file(os.path.join(root, file),"H:/TFIDFgujarati Text summarization dataset/economics/"+file)
                #load_file(os.path.join(root, file),"G:/Movies/post_1_summary/"+file)
# total = sum(total_sen)
# print("Sum of all elements in given list: ", total)

this is text 
પેટ્રોલ-ડીઝલની નવી કિંમતો (Petrol Diesel Price) જાહેર થઈ ચૂકી છે. સરકારી ઓઇલ કંપનીઓએ સામાન્ય જનતાને રાહત આપતાં પેટ્રોલ-ડીઝલના ભાવમાં ઘટાડો કર્યો છે. ઓઇલ કંપનીઓએ 15 દિવસ પેટ્રોલ-ડીઝલની કિંમત સ્થિર રાખ્યા બાદ તેના ભાવમાં ઘટાડો કર્યો છે. આજે પેટ્રોલની કિંમત (Petrol Price Today)માં 16 પૈસા પ્રતિ લીટર અને ડીઝલ (Diesel Price Today)ની કિંમતમાં 14 પૈસા પ્રતિ લીટરનો ઘટાડો કર્યો છે. આ ઘટાડા બાદ દિલ્હીમાં એક લીટર પેટ્રોલનો ભાવ 90.40 રૂપિયા પ્રતિ લીટર, તો ડીઝલ 80.73 રૂપિયા પ્રતિ લીટર થઈ ગયો છે. મૂળે, આંતરરાષ્રીો ય માર્કેટમાં કાચા તેલની કિંમતોમાં ઘટાડાની અસર સ્થાનિક બજારમાં જોવા મળી છે.

માર્ચ મહિનામાં ત્રણ વાર સસ્તા થયા હતા પેટ્રોલ-ડીઝલ
માર્ચ 2021માં પેટ્રોલ અને ડીઝલના ભાવમાં ત્રણ વાર ઘટાડો કરવામાં આવ્યો હતો. 30 માર્ચે પેટ્રોલ 22 પૈસા અને ડીઝલ 23 પૈસા પ્રતિ લીટર સસ્તું થયું હતું. આ પહેલા 24 અને 25 માર્ચે પેટ્રોલ ડીઝલ સસ્તા થયા હતા.


this is text 
કોરોના મહામારીના (Coronavirus) કારણે સમગ્ર વિશ્વની આર્થિક ગતિવિધિઓ ઉપર રોક લાગી ગઈ હતી. ભારત સહિત મોટાભાગના દેશોના અર્થતંત્રની હાલત ખરાબ થ

this is text 
કેપિટલ માર્કેટ અને સેકન્ડરી માર્કેટમાં સમયાંતરે જરૂરી ફેરફારો કરવા નિયામક સંસ્થા સેબી કોરોનાકાળમાં પણ કટિબદ્ધ છે. પોસ્ટ આઈપીઓ લોક ઈનનો સમયગાળો સેબી ઘટાડીને પ્રમોટરોને રાહત આપવા અંગે વિચારી રહી છે.

જાહેર ભરણાં(IPO) બાદના ત્રણ વર્ષ સુધી કંપનીના પ્રમોટર પોતાનું હોલ્ડિંગ એટલે કે પોતાની હિસ્સેદારી 20%થી નીચે ન લઈ જઈ શકે તેવો સેબીનો કડક નિયમ રીટેલ રોકાણકારોને ધ્યાનમાં રાખીને લેવામાં આવ્યો હતો. પરંતુ હવે બજારમાં લિક્વિડિટી અને રીટેલ પાર્ટિસિપેશન જોઈને સિક્યોરિટી એન્ડ એક્સચેન્જ બોર્ડ ઓફ ઈન્ડિયા(SEBI) આ સમયગાળો ઘટાડીને ન્યૂનતમ 1 વર્ષ કરવા જઈ રહી છે.

સેબીએ ICDR (Issue of Capital and Disclosure Requirements) નિયમોમાં ઢીલ મુકવાનો નિર્ણય કર્યો છે. આ નિયમ હળવો બનતા પ્રમોટરો માટે આઈપીઓ બાદના કડક નિયમપાલનથી છુટકારો મળશે. 20%થી વધારાના અને પ્રી-આઈપીઓ નોન પ્રમોટર શેરહોલ્ડિંગ માટેના નિયમનો સમયગાળો પણ 1 વર્ષના લોકઈનથી ઘટાડીને 6 મહિના કરશે.

આ સિવાય સેબીએ જ એક નિવેદનમાં કહ્યું છે કે તેઓ પ્રમોટરોની પરિભાષામાં પણ ફેરફાર કરવા જઈ રહ્યાં છે. વ્યાખ્યા બદલવા પાછળનું કારણ આપતા સેબીએ કહ્યું કે હવે પ

this is text 
સોમવારે સેન્સેકસ (Sensex) અને નિફ્ટી (Nifty)માં મસમોટું ગાબડું પડી ગયું હતું. શેરજારમાં કડાકા પાછળ કોરોના વાયરસનાં કારણે લોકડાઉન (Lockdown)નો ડર જવાબદાર ગણવામાં આવે છે. અલબત્ત ભારતની જેમ અન્ય આંતરાષ્ટ્રીય બજારમાં ગાબડું પડ્યું નહોતું. આર્થિક સલાહકારોએ તેમના ક્લાયન્ટને આંતરરાષ્ટ્રીય ઇક્વિટીમાં 10થી 20 ટકા જેટલો પોર્ટફોલિયો સંભાળી રાખવાની સલાહ આપી રહ્યા છે. ડોમેસ્ટિક માર્કેટમાં કડાકા સામે આ આંતરરાષ્ટ્રીય પોર્ટફોલિયો હેજ તરીકે કામ કરશે.

વૈશ્વિક ફંડ મોટા બજારોના વધુ તક હોય તેવી કંપનીઓમાં રોકાણ કરે છે. આ નિર્ણય પાછળ સસ્તી ઍક્સેસ અને વેલ્યુએશન સહિતના પાસાઓ પણ ધ્યાને લેવાય છે. આવી સ્થિતિમાં ઇન્ટનેશનલ મ્યુચ્યુઅલ ફંડમાં શું કામ રોકાણ કરવું જોઈએ? તેવા પ્રશ્નો ઉઠે છે. જેથી અહી ત્રણ કારણ આપવામાં આવ્યા છે.

ભારતમાં ઓછા સંકળાયેલ બજારો

આંતરરાષ્ટ્રીય બજારો સાથે ભારતીય બજારોનો સબંધ ઓછો છે. મીરા એસેટ મ્યુચ્યુઅલ ફંડના અહેવાલ મુજબ ભારતીય બજારોનો યુએસ સાથે 0.16%, યુરોપ સાથે 0.32% અને ચીન સાથે 0.38%થી ઓછો સંબંધ છે. જેથી આ બજારોમાં રોકાણ કરતા આંતરરાષ્ટ્રીય મ્યુચ્યુઅલ ફંડ્સમાં રોકાણ કરવાથી તમા

this is text 
હાલના દિવસોમાં આઇપીઓ શેર માર્કેટ (Share market)માં ધૂમ મચાવી રહ્યા છે. અત્યાર સુધીમાં જેટલા પણ IPO લોન્ચ થયા છે, તેમાં લગભગ બધાએ રોકાણકારોને ઉત્તમ રિટર્ન આપ્યું છે. છેલ્લું વર્ષ એટલે કે 2020 ભારતીય આઈપીઓ માર્કેટ માટે શાનદાર સાબિત થયું છે. આશરે 15 મોટી કંપનીઓએ પોતાના આઈપીઓથી આશરે 31,000 કરોડ રૂપિયા ઉભા કર્યા છે. વર્ષ 2021માં પણ IPO માર્કેટમાં જોરદાર હલચલ જોવા મળી રહી છે. પ્રાઇમરી માર્કેટમાં તેજીને ધ્યાનમાં રાખીને, કંપનીઓ એક પછી એક પોતાના આઈપીઓ લાવી રહી છે. જો અત્યાર સુધી તમે આઈપીઓમાં રોકાણ કરીને પૈસા કમાવવાનું ચુકી ગયા છો, તો પછી આ 5 IPO એપ્રિલમાં આવવાના છે (IPO in April 2021). ચાલો જાણીએ તેમના વિશેની વિગતો.

7 Aprilએ રીઅલ એસ્ટેટ કંપની મેક્રોટેક ડેવલપર્સનો આઈપીઓ આવી ગયો છે. એપ્રિલ મહિનામાં, રૂ. 18,000 કરોડના આઈપીઓ બજારમાં આવી શકે છે. હવે જે 5 આઇપીઓ આવી રહ્યા છે તે છે 1. સેવન આઇલેન્ડ શિપિંગ (Seven Islands Shipping), 2. સોના બીએલડબ્લ્યુ પ્રેસિશન ફોર્સીંગ (Sona BLW Precision Forgings ), 3. આધાર હાઉસિંગ ફાઇનાન્સ (Aadhar Housing Finance), Krishna. કૃષ્ણ ઇન્સ્ટિટ્યૂટ ઓફ મેડિકલ સ

this is text 
આંતરરાષ્ટ્રીય મુદ્રાકોષે (IMF)ભારતનો આર્થિક વૃદ્ધિદર (India's Growth)ઝડપથી વધીને 12.5 ટકા પર પહોંચવાનો અંદાજ લગાવ્યો છે. આઈએમએફનું કહેવું છે કે ભારત વૃદ્ધિદરના મામલે ચીનને પાછળ રાખી દેશે. ચીન એકમાત્ર મોટી અર્થવ્યવસ્થા રહ્યું છે જેનો વૃદ્ધિદર 2020માં કોરોના મહામારી દરમિયાન પણ સકારાત્મક રહ્યો હતો. આઈએમએફે પોતાના વાર્ષિક વૈશ્વિક આર્થિક પરિદશ્યમાં (Global Economic Outlook)કહ્યું કે 2020માં ભારતીય અર્થવ્યવસ્થાની (Indian Economy)વૃદ્ધિદર 6.9 ટકાની આસપાસ આવી જશે. તમને જણાવી દઈએ કે મુદ્રાકોષે વિશ્વબેંક (World Bank)સાથે થનાર વાર્ષિક બેઠક પહેલા આ રિપોર્ટ જાહેર કર્યો છે.

આઈએમએફે કહ્યું કે 2020માં ભારતીય અર્થવ્યવસ્થામાં રેકોર્ડ 8 ટકાની ગિરાવટ આવી હતી. હવે આ વર્ષે વૃદ્ધિદર 12.5 ટકા રહેવાનો અંદાજ છે. જે વર્તમાન સ્થિતિમાં ઘણો સારો છે. ચીનનો (China)વૃદ્ધિદર 2021માં 8.6 ટકા અને 2022માં 5.6 ટકા રહેવાનો અંદાજ લગાવવામાં આવ્યો છે.

ચીનનો ગત વર્ષે વૃદ્ધિદર 2.3 ટકા રહ્યો અને તે કોવિડ-19 મહામારી દરમિયાન પણ સકારાત્મક આર્થિક વૃદ્ધિ દર મેળવનાર દુનિયાનો એકમાત્ર મોટો દેશ રહ્યો હતો.

આઈએમએફના મુખ્ય અ

In [13]:
from rouge.rouge import rouge_n_sentence_level
from rouge import rouge_n_summary_level
from rouge import rouge_l_summary_level
from rouge import rouge_w_sentence_level
from rouge import rouge_w_summary_level

In [14]:
from statistics import mean
def Average(lst):
    return mean(lst)

In [41]:
import codecs
import string
import re
from bs4 import BeautifulSoup  
from nltk.tokenize import word_tokenize
rouge1=[]
rouge2=[]
rougel=[]
rougew=[]

class ShortText:
    def __init__(self, my_id, human_label, summary, short_text):
        self.id = my_id         
        self.human_label = human_label    
        self.summary = summary 
        self.short_text = short_text
    def __str__(self):
        '''
        For printing purposes.
        '''
        return '%d\t%d\t%s\t%s' % (self.id, self.human_label, self.summary, self.short_text)

def load_file(filename,reference):
    tokenize_words=[]
    tokenize_words1=[]
    
    #file1 = open(reference, "r")
    #retrieve the original text 
    with codecs.open(filename, encoding='utf-8') as f:
        data = f.read()
    #print(data)
    
    for tokenize in indic_tokenize.trivial_tokenize(data):
            tokenize_words.append(tokenize)
        # Convert list of strings to string
    originalfull_str = convert_list_to_string(tokenize_words)
    senlateron=[]
    sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
    i=1
    with codecs.open(reference+"a2_"+str(i)+".txt", encoding='utf-8') as f1:
        data1 = f1.read()
    i=i+1
    for tokenize1 in indic_tokenize.trivial_tokenize(data1):
            tokenize_words1.append(tokenize1)
        # Convert list of strings to string
    originalfull_str1 = convert_list_to_string(tokenize_words1)
    senlateron1=[]
    sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
    _, _, rouge_1 = rouge_n_summary_level(tokenize_words, tokenize_words1, 1)
    rouge1.append(rouge_1)
    #print('ROUGE-1: %f' % rouge_1)
    _, _, rouge_2 = rouge_n_summary_level(tokenize_words, tokenize_words1, 2)
    #print('ROUGE-2: %f' % rouge_2)
    rouge2.append(rouge_2)
    _, _, rouge_l = rouge_l_summary_level(tokenize_words, tokenize_words1)
    #print('ROUGE-L: %f' % rouge_l)
    rougel.append(rouge_l)
    _, _, rouge_w = rouge_w_summary_level(tokenize_words, tokenize_words1)
    #print('ROUGE-W: %f' % rouge_w)
    rougew.append(rouge_w)
    print(rouge1)
    average1 = Average(rouge1)
    print("average of rouge1",average1)
    print(rouge2)
    average2 = Average(rouge2)
    print("average of rouge2",average2)
    print(rougel)
    averagel = Average(rougel)
    print("average of rougel",averagel)
    print(rougew)
    averagew = Average(rougew)
    print("average of rougew",averagew)
    #for t,t1 in zip(sentences,sentences1):
        #print(t.split())
        #print(t1.split())
        
    #use beautifulsoup to get tag attributes and elements
    #soup = BeautifulSoup(data)
#     numid=soup.find_all('num')
#     tit=soup.find_all('title')
#     tags = soup.find_all('desc')
#     narr = soup.find_all('narr')
    #docno=soup.find_all('docno')
    #print("hello1")
    #text=soup.find_all('text')
    #store in a dictionary with ShortText Instances as values
    instances = {}
#     my_id = 0
#     for n,tit in zip(docno,text):
#         #print("hello2")
#         tit=tit.get_text()
#         tit=str(tit)
#         s = re.sub('[!#?,.:";]', '', tit)
#         tokenize_words=word_tokenize(s)
#         #print(tokenize_words)
#         how_many_times=2
#         for i in range(how_many_times):
#             #print("in loop")
#             first(tokenize_words)
#             modlist=second(tokenize_words)
#         #print(modlist)
#         originalfull_str = convert_list_to_string(modlist)
        
#         n=str(n)
        
#         #print(type(t))
#         #li=convert(no_specials_string)
#         file1.write("<DOC>\n"+n+"\n<TEXT>\n"+originalfull_str+"</TEXT>\n</DOC>")
#         list1.clear()
#         list2.clear()

    return instances

In [42]:
for root, dirs, files in os.walk("H:/TFIDFgujarati Text summarization dataset/economics/"):    
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(root, file), 'r') as f:
                #print(os.path.join(root, file))
                load_file(os.path.join(root, file),"C:/Users/HM-AM/Documents/GitHub/Gujarati-Textsummarization-corpus/gujarati Text summarization dataset/economics/annotator2/")

[0.8987341772151899]
average of rouge1 0.8987341772151899
[0.8063492063492064]
average of rouge2 0.8063492063492064
[0.8987341772151899]
average of rougel 0.8987341772151899
[0.38099204140616605]
average of rougew 0.38099204140616605
[0.8987341772151899, 0.6000000000000001]
average of rouge1 0.7493670886075949
[0.8063492063492064, 0.19291338582677164]
average of rouge2 0.499631296087989
[0.8987341772151899, 0.6000000000000001]
average of rougel 0.7493670886075949
[0.38099204140616605, 0.20866101951127022]
average of rougew 0.29482653045871815
[0.8987341772151899, 0.6000000000000001, 0.6195965417867436]
average of rouge1 0.7061102396673112
[0.8063492063492064, 0.19291338582677164, 0.2225433526011561]
average of rouge2 0.4072686482590447
[0.8987341772151899, 0.6000000000000001, 0.6195965417867436]
average of rougel 0.7061102396673112
[0.38099204140616605, 0.20866101951127022, 0.2612625048820209]
average of rougew 0.2836385219331524
[0.8987341772151899, 0.6000000000000001, 0.6195965417867

[0.8987341772151899, 0.6000000000000001, 0.6195965417867436, 0.46567164179104475, 0.5456821026282854, 0.5850340136054422, 0.19282868525896413, 0.30966239813736907, 0.5325581395348837, 0.5174672489082969, 0.5780346820809249, 0.3090128755364807, 0.5022026431718061]
average of rouge1 0.512037319204264
[0.8063492063492064, 0.19291338582677164, 0.2225433526011561, 0.19740777666999004, 0.1882057716436637, 0.22646657571623463, 0.13556618819776714, 0.19347319347319347, 0.2564102564102564, 0.2210065645514223, 0.23478260869565218, 0.1878453038674033, 0.1920529801324503]
average of rouge2 0.25038639724116674
[0.8987341772151899, 0.6000000000000001, 0.6195965417867436, 0.46567164179104475, 0.5456821026282854, 0.5850340136054422, 0.19282868525896413, 0.3084982537834691, 0.530232558139535, 0.5152838427947597, 0.5751445086705201, 0.3090128755364807, 0.5022026431718061]
average of rougel 0.5113786034140185
[0.38099204140616605, 0.20866101951127022, 0.2612625048820209, 0.21396240959374824, 0.2240645619

[0.8987341772151899, 0.6000000000000001, 0.6195965417867436, 0.46567164179104475, 0.5456821026282854, 0.5850340136054422, 0.19282868525896413, 0.30966239813736907, 0.5325581395348837, 0.5174672489082969, 0.5780346820809249, 0.3090128755364807, 0.5022026431718061, 0.48678414096916295, 0.44357976653696496, 0.41305998481397116, 0.3779160186625194, 0.4979919678714859, 0.44739336492890996]
average of rouge1 0.49069528386518135
[0.8063492063492064, 0.19291338582677164, 0.2225433526011561, 0.19740777666999004, 0.1882057716436637, 0.22646657571623463, 0.13556618819776714, 0.19347319347319347, 0.2564102564102564, 0.2210065645514223, 0.23478260869565218, 0.1878453038674033, 0.1920529801324503, 0.27593818984547464, 0.22807017543859648, 0.19923954372623576, 0.19626168224299068, 0.22736418511066395, 0.1994301994301994]
average of rouge2 0.241122481048912
[0.8987341772151899, 0.6000000000000001, 0.6195965417867436, 0.46567164179104475, 0.5456821026282854, 0.5850340136054422, 0.19282868525896413, 0.3

In [27]:
# text="""વાઈવા આપતી વખતે, એપ્રનના પહેલા બે બટન ખોલી નાંખતી હશે. એમબીબીએસમાં મારી જ સાથે ભણતા એક મેડિકલ સ્ટુડન્ટે અમારા ક્લાસની ટોપર અને સૌથી હોંશિયાર છોકરી વિશે કરેલી ટિપ્પણી મને આજે પણ યાદ છે. આપણી આસપાસ એવા અઢળક લોકો છે જે એવું માને છે કે સફળતાનો રસ્તો પુરુષ કરતા સ્ત્રી માટે વધારે ટૂંકો હોય છે.  
# પુરુષની સફળતાને એના સાહસ, સમર્પણ અને મહેનત સાથે જોડવામાં આવે છે  જ્યારે  સ્ત્રીની સફળતાને શક્યતાઓ સાથે. એ કોઈ કલાકાર હોય, જર્નલીસ્ટકે ઓફિસ વર્કર. કોઈપણ ક્ષેત્રમાં એક સ્ત્રી જ્યારે અચાનક લાઈમ-લાઈટમાં આવે છે, ત્યારે એની પ્રસિદ્ધિનો યશ એની પ્રતિભા કરતા એની સુંદરતા, એના ફિઝીક અને એની 'ઉદારતા' ને  આપવામાં આવે છે. 
# સફળતાની સીડીઓ ચડી રહેલી સ્ત્રીને પાછળ ધકેલવા અને એણે મેળવેલી સિદ્ધિઓનો અનાદર કરવા માટે અન્ય કોઈ સ્ત્રી કે પુરુષ પાસે રહેલું  સૌથી  જુનું અને ધારદાર  હથિયાર એટલે  'કેરેક્ટર એસેસીનેશન.' એ ઈર્ષા હોય  કે ઈન્ફીરીયોરીટી, રેસમાં રહેલી વ્યક્તિને જ્યારે એવું પ્રતીત થાય છે કે આ સ્ત્રીને હવે 'મેરીટ' થી નહીં હરાવી શકાય ત્યારે વાત ચારિત્ર્ય પર આવે છે.
# આ ફેમિનિઝમ નથી. કારણકે ફેમિનિઝમ તો સ્ત્રીઓને સમાન હક અને તક અપાવવા માટે હોય છે. ફેમિનિઝમ સ્ત્રીઓને લિબરેટ કરવા માટે હોય છે. જ્યારે આ સમસ્યા તો સ્ત્રીઓના લિબરેશન પછીની છે. 
# ફેમિનિઝમના સહારે અને ભરોસે રહેવાને બદલે એક સ્ત્રી જ્યારે પોતાના હકની સિદ્ધિ અને સફળતા, પોતાની મહેનત અને ટેલેન્ટના જોર પર ઝૂંટવી લે છે ત્યારે વાત એના ચારિત્ર્ય પર આવે છે. 
# ઈસ્ટ્રોજન સામેની દોડમાં જ્યારે ટેસ્ટોસ્ટેરોન ઘવાય છે, ત્યારે વાત એના ચારિત્ર્ય પર આવે છે. આજ સુધી ફક્ત 'સુંદરતા' ને જ એક સ્ત્રીની USP ગણનારા સમાજની આંખો જ્યારે એની બુદ્ધિ અને પ્રતિભાથી અંજાઈ છે, ત્યારે વાત એના ચારિત્ર્ય પર આવે છે.
#  સ્ત્રીના સક્સેસને એની સેક્સ્યુઆલીટી સાથે જોડી દેવામાં આવે છે. 
# સ્ત્રીની સફળતા 'સામાજિક અપચો' સર્જે છે. આપણે ત્યાં પોલીટીક્સ કે ક્રિકેટ કરતા પણ વધારે કશુંક ચર્ચાતું હોય, તો એ એક સ્ત્રીની સફળતા છે. દરેક પાસે પોતાના અભિપ્રાયો અને તારણો છે. 
# જે સમયે કોઈપણ જાતના જજમેન્ટ પાસ કર્યા વગર એક આગળ વધી રહેલી સ્ત્રીની સફળતાને આપણે 'એઝ ઈટ ઈઝ' સ્વીકારતા થઈ જશું, ત્યારે આપણે સિવિલાઈઝ્ડ કહેવાશું. ખ્યાતિ, લોકપ્રિયતા કે પ્રસિદ્ધિ મેળવવા માટે દરેક સ્ત્રીએ 'કોમ્પ્રોમાઈઝ' કરવું જ પડે છે,  એવું માનનારા લોકો પોતાની સમજણ સાથે કોમ્પ્રોમાઈઝ કરી રહ્યા છે. સ્ત્રીની સફળતા એણે કરેલું સમાધાન નથી, એણે કરેલી સાધના છે. 
# હા, એણે કશુંક ઓફર કર્યું છે. પોતાની મહેનત, પ્રતિભા, સમર્પણ અને સાહસ. એણે કોમ્પ્રોમાઈઝ પણ કર્યું છે. કુટુંબને આપેલા સમય સાથે, પોતાના મોજ-શોખ અને સુખ-સગવડો સાથે.
# એક સ્ત્રી જ્યારે પોતાના 'કમ્ફોર્ટ ઝોન' માંથી બહાર નીકળે છે, ત્યારે એની સફળતા બીજા અનેક લોકો માટે 'અનકમ્ફોર્ટેબલ' બની જતી હોય છે. 
# -ડૉ. નિમિત્ત ઓઝ"""

# text="""All said and done, મને મળી રહેલા પર્સનલ મેસેજીસની સંખ્યા 'અનમેનેજબલ' થઈ રહી છે. મેઈલ કે મેસેજ દ્વારા જે રીતે અને જે સંખ્યામાં લોકો પોતાની ઉદાસી કબુલી રહ્યા છે, પોતાને બચાવવા માટે રીક્વેસ્ટ કરી રહ્યા છે, એ જોઈને હું સ્તબ્ધ છું. મેં ક્યારેય નહોતું વિચાર્યું કે સોશિયલ મીડિયા પર 'ફીલીંગ હેપ્પી' કે 'ફિલિંગ એક્સાઈટેડ' લખીને પોતાના હસતા ફોટોઝ શેર કરનારા લોકો અંદરથી આટલા બધા ઉદાસ હશે. 
# So I thought, Let me share something from my personal experience.  કદાચ કોઈને ઉપયોગી થાય. 
# ૧. સૌથી પહેલા તાત્કાલિક કોઈ થેરાપીસ્ટ, મનોચિકિત્સક કે કાઉન્સેલરને મળો. સારવારનો પહેલો તબક્કો સ્વીકાર છે. 'હા, છીએ ઉદાસ' એવું વટથી જ્યાં સુધી સ્વીકારશું નહીં, ત્યાં સુધી એમાંથી બહાર નહીં આવી શકાય. ASAP  દવા કે કાઉન્સેલિંગ સેશન શરૂ કરો. ડિપ્રેશનની દવા અસર કરતા ૧૫-૨૦ દિવસ લાગે છે. ત્યાં સુધી 'નો સ્યુસાઈડ કોન્ટ્રેક્ટ' સાઈન કરો. તમારી જાતને અને થેરાપીસ્ટને પ્રોમીસ કરો કે 'ક્ષણિક ઉદાસીનો ભોગ બનીને તમે મૂર્ખામીભર્યું કોઈ કાયમી પગલું નહીં ભરો.' યાદ રાખો, મૃત્યુ ઈરરીવર્સીબલ છે. 
# ૨. 'મેન્ટલ ડાયેરીયા' બંધ કરવા માટે મગજને બિનજરૂરી અને વાહિયાત માહિતીઓ પીરસવાનું બંધ કરો. એન્ડ ઈન પ્રોસેસ ઓફ ડૂઇંગ સો, સૌથી પહેલા સોશિયલ મીડિયા અને ન્યુઝ ચેનલ્સ બંધ કરો. ફેસબુક, ઇન્સ્ટા, વ્હોટ્સ-એપ પરથી થોડો સમય બ્રેક લઈ લો. સોશિયલ મીડિયા એ ડિપ્રેશનનું સૌથી મોટું કારણ છે કારણકે Every time we sign into it, unknowingly we are landing into the zone of comparison. કોઈપણ સ્તરે 'કમ્પેરીઝન' આપણા માનસિક સ્વાસ્થ્ય માટે જીવલેણ છે. 
# સંપર્ક માટેના ત્રણ શ્રેષ્ઠ રસ્તાઓ જ ખુલ્લા રાખો. 'રૂટીન પ્રાયોરીટી' કામ હોય તો ઈ-મેઈલ, સેમી-અરજન્ટ હોય તો ટેક્સ્ટ મેસેજ (SMS) અને બહુ અરજન્ટ હોય તો ફોન કોલ. 
# સોશિયલ મીડિયા પરથી બ્રેક લેવાથી આપણે કશું જ ગુમાવવાના નથી. આપણે કોઈ અગત્યની વસ્તુ કે વ્યક્તિને મિસ નહીં કરીએ. એન્ડ બીલીવ મી, નો-બડી ઇઝ ગોઈંગ ટુ મિસ અસ ઓલ્સો. (I took a break for 3 months from all social media last year). એ દરમિયાન જે માનસિક શાંતિનો અનુભવ થાય છે, એ પ્રાઈસ-લેસ છે. ટ્રાય ઇટ યોરસેલ્ફ.  
# બ્રેક ન લો તો પણ, રીસર્ચ એવું કહે છે કે સોશિયલ મીડિયા પર ૩૦ મીનીટ કે ૧ કલાકથી વધારે સમય ન પસાર કરો. It will cause you depression. 
# ૩. મેડિટેશન. મેડિટેશન. મેડિટેશન. આજ નહીં તો કાલ નહીં તો વર્ષો પછી, મેડિટેશન સિવાય ઉદ્ધાર નથી. We all are constantly surrounded by people full of shit, and only Meditation can rescue us. 
# પ્રોબ્લેમ એ છે કે આપણે જેટલા યુદ્ધને ગ્લોરીફાય કર્યા છે, એટલા બુદ્ધને કર્યા હોત તો આટલું બધું હ્યુમન સફરીંગ ન હોત. 
# મેડિટેશન અંગે તમારા મનમાં જેટલી ખોટી માન્યતાઓ હોય, એટલી કાઢી નાખો. મેડિટેશન એ થોટલેસનેસ નથી. તમારે કોઈ શીબીર કે ધ્યાન ગુરુ પાસે જવાની જરૂર નથી. કોઈ જંગલમાં કે પર્વત પર જવાની જરૂર નથી. એ તમે ઘેર બેઠા કરી શકો છો અને એ પણ એકલા. 
# 'ગાઇડેડ મેડિટેશન' માટે તમારા ફોનમાં 'Calm' કે 'Headspace' એપ ડાઉનલોડ કરો. (હું હેડસ્પેસ યુઝ કરું છું એન્ડ આઈ એમ લવીંગ ઇટ.'
# મેડિટેશન અંગે બેઝીક્સ જાણવું હોય તો આટલું વાચી જાવ. ધિસ વિલ હેલ્પ યુ અ લોટ. https://www.thinklessandgrowrich.com/meditation-for-beginners-the-complete-guide/
# ૪. પુસ્તકો વાચો. Your brain deserves much better content than non-sense forwards, silly jokes, and good morning messages.  Few good reads are : 
# 1. Kick the thinking habit by Richard Paterson 
# 2. The subtle art of not giving a F*ck  by Mark Manson 
# 3.  The power of Now by Eckhart Tolle 
# 4.  Any book by Dalai Lama
# 5.  Any book by Mahatma Gandhi 
# 6.  Any book on Buddha and Buddhism 
# 7.  વિનોબાજીના ગીતા પ્રવચનો 
# (ફોનમાંથી નકામી એપ ડીલીટ કરીને એમેઝોન કિન્ડલ ઈન્સ્ટોલ કરો. See, How life changes !)
# ૫. ઈકો-થેરાપી : જંગલ, પર્વત, નદી, દરિયો અને ખુલ્લું આકાશ. આ સૌથી મોટા એન્ટી-ડીપ્રેસન્ટ્સ છે. Nature heals the soul.  આટલે બધે દૂર ન જઈ શકાય, તો ઘરની આસપાસ રહેલા ગાર્ડન કે પાર્કમાં જાવ. વૃક્ષો સાથે સમય વિતાવો. ક્યારેય ડિપ્રેશન નહીં આવે. હવે, તો આખી ફોરેસ્ટ મેડિસિનની બ્રાંચ ડેવલપ થઈ રહી છે, જે કહે છે કે ‘If you hug a tree, It will make you happy.'  (https://qz.com/1208959/japanese-forest-medicine-is-the-art-of-using-nature-to-heal-yourself-wherever-you-are/)
# ૬. મનમાં આવતા નેગેટીવ, નિરાશાજનક અને વાહિયાત વિચારોનું ઉત્સર્જન કરો. ડિપ્રેશન એ બીજું કાંઈ નથી પણ અવ્યક્ત રહી ગયેલા એટેન્શન સીકિંગ વિચારોનો આપણી બુદ્ધિ સામે થયેલો બળવો છે. કેથાર્સીસ કે વિરેચન માટેનું કોઈ એક માધ્યમ શોધી કાઢો. બેસ્ટ વે ઈઝ ટુ રાઈટ ડાઉન યોર ફીલિંગ્સ, ઇમોશન્સ, ફીઅર્સ એન્ડ ઈનસિક્યોરીટીઝ. 
# દરેક નબળા અને હેરાન કરતા વિચારોની કાગળ પર ઉલટી કરી નાખો. પછી એ કાગળ ફાડી કે બાળી નાખો. It works ! 
# ૭. ગ્રેટીટ્યુડ ટેકનીક : રોજ સવારે ઉઠીને સૌથી પહેલા એક કાગળ પર એવી ત્રણ વસ્તુઓ લખો, જેના માટે તમે ઈશ્વરના આભારી છો. એ કાંઈ પણ હોય શકે છે. જેમકે તમે જીવો છો, તમારું ફેમીલી સ્વસ્થ છે, તમારા સંતાનો સુખી છે, તમારા ઘરની ઉપર છત છે, ઘરમાં ખાવાનું છે, એની ડેમ થીંગ.
# એક ગ્લોબલ પોવર્ટી ફેક્ટ : આ વિશ્વની અડધી જનસંખ્યા એટલે કે ૩ અબજથી વધારે લોકો રોજના ૨.૫ ડોલર (એટલે કે બસોથી પણ ઓછા રૂપિયામાં) પોતાનું ગુજરાન ચલાવે છે. આટલા રૂપિયાનું તો આપણું વાહન દરરોજનું પેટ્રોલ પી જતું હશે. 
# યુનિસેફના આંકડા પ્રમાણે વિશ્વના ૨૨,૦૦૦ બાળકો દરરોજ ભૂખમરાથી મૃત્યુ પામે છે.  
# આવું બધું વિચારશું, તો મરવાની ઈચ્છા નહીં થાય. 
# ૮. ભલે આળસ આવે, પણ પરાણે કસરત કરો. એક્સરસાઈઝનો સૌથી મોટો ફાયદો માનસિક સ્વાસ્થ્ય પર થાય છે. જીમ કરો, સાયકલીંગ, રનીંગ, સ્કીપીંગ, સ્વીમીંગ કાંઈ પણ કરો. કાંઈ નહીં તો ગો ઓન અ લોંગ વોક. એ કરવાથી શરીરમાં એન્ડોર્ફીન્સ અને ડોપામીન રીલીઝ થાય છે. ધિસ વિલ મેક યુ ફીલ ગુડ. 
# ૯. મ્યુઝીક આત્માનો ખોરાક છે. હાવ કેન યુ ઈગ્નોર ધેટ ? તમારા હેડ-ફોન્સ લગાડો અને તમારા ગમતા પ્લે-લીસ્ટમાં ખોવાઈ જાવ.
# ૧૦. ધ લીસ્ટ ઈઝ એન્ડ-લેસ. હજી ઘણું બાકી છે. આઈ પ્લાન ટુ સ્ટાર્ટ અ સીરીઝ ઓફ આર્ટીકલ્સ, પોડકાસ્ટ એન્ડ વિડિયોઝ ઓન મેન્ટલ હેલ્થ. ઈફ યુ આર રીઅલી ઈન્ટરેસ્ટેડ, મારા બ્લોગ https://drnimitoza.in/ પર જઈને સબસ્ક્રાઈબ કરો. 
# યાદ રાખો, We all deserve a better life. And it is just a decision away.  
# Take care.  Love and gratitude.
# -ડૉ. નિમિત્ત ઓઝાા"""

text="""અમદાવાદના એક જાણીતા અખબારમાં ચાર કોલમનું બોક્સ એક છોકરી વિશે છપાયેલું. એના પિતા સાથેના કોઈક વાંધાને કારણે એના ચારિત્ર્ય ઉપર સાચા-ખોટા છાંટા ઉડાડવામાં આવેલા. સવારના પહોરમાં અખબાર વાંચતાની સાથે જ એ છોકરીને લાગેલું કે એનો બોયફ્રેન્ડ હવે એની સામે પણ નહીં જુએ. "છાપામાં છપાઈ ગયું..." એ કેટલી ભયાનક વાત હોઈ શકે, આજથી અઢી દાયકા પહેલાં ! સવારના દસ વાગ્યા પહેલાં નહીં ઉઠનારો એનો બોયફ્રેન્ડ સાડા આઠ વાગ્યામાં ખમણ અને ગાંઠિયા લઈને એના ઘરે પહોંચ્યો... સહુએ સાથે નાસ્તો કર્યો એટલું જ નહીં, એણે હસીને કહ્યું, "લખ્યા કરે... મારે માટે તું વધારે સાચી કે આ લખનાર ?"
આજથી અઢી દાયકા પહેલાં, એકવાર ડીવોર્સ થઈ ચૂક્યા હોય એવી છોકરી સાથે લગ્ન કરનારો કુંવારો છોકરો એને એકવાર પણ પૂછે નહીં, "એક્ચ્યુઅલી શું થયેલું ?" તો એને માટે સન્માન થયા વગર રહે ? જોકે, ત્યારે સન્માનનો અર્થ પ્રેમ થાય કે નહીં, એ વિશે હજી નિર્ણય કરવાની મારી ઉંમર નહોતી અને મનઃસ્થિતિ પણ નહીં.
"એક વર્ષથી આપણે સાથે ફરીએ છીએ. મારા મા-બાપ રોજ પૂછે છે... હવે આપણે પરણી જવું જોઈએ એવું તને નથી લાગતું ?" ઠંડીના દિવસોમાં ગાંધીગ્રામ સ્ટેશન ઉપર, હાથમાં ચાના કપ પકડીને પસાર થતી ટ્રેનના અવાજમાં એણે મને પૂછ્યું હતું ! જો હું ના પાડું, તો ટ્રેનના અવાજમાં એ ના દબાઈ જાય... એવી એની ગણતરી હતી જે એણે મને પછીથી કહેલું !
          ત્યારે હા કે ના કહેવાનો સવાલ જ નહોતો... એની સાથે જીવવાની ઈચ્છા હતી, પણ લગ્ન વિશે મારા મનમાં બહુ મોટો અવઢવ હતો, કારણ એ, કે મારા પ્રથમ લગ્નમાં હું અપમાન, દાદાગીરી અને ડોમેસ્ટીક વાયોલેન્સનો ભોગ બની ચૂકી હતી. ઓગણીસ વર્ષે, જિંદગી શરૂ થાય એ પહેલાં એરેન્જ મેરેજ પૂરાં થઈ ચૂક્યાં હતાં ! હું કદાચ મારા સમયથી બહુ આગળ જીવી હોઈશ. આજના છોકરાંઓને કમિટમેન્ટનો વિરોધ કરતા જોઉં ત્યારે લાગે કે મારા મનમાં આ ગૂંચવણ 27 વર્ષ વહેલી જન્મી ગઈ હતી ! એના મનમાં ક્લેરીટી હતી. પરણવું છે ! ને, એ પરણ્યો... મારી સાથે! એના મિત્રો, પરિવારજનો અને બીજા ઘણાએ એને આવું નહીં કરવા સમજાવ્યો હતો, પણ એ સ્વભાવે સાહસિક છે... આજે પણ !
         નામ એનું સંજય વૈદ્ય. વ્યવસાયે ફોટોગ્રાફર, ને સ્વભાવે અલગારી. ત્યારે ગુજરાત સમાચારમાં નોકરી કરતો, પણ નોકરી એનો સ્વભાવ નથી. સાચું પૂછો તો એ લગ્ન કરવા માટેનું મટીરીયલ જ નથી. જોકે એ મિત્ર ખૂબ સારો. કલાકો સાંભળી શકે. આપણી છટકી હોય ત્યારે શાંત રહી શકે. ફોન ઉપર હું 'હલો' કહું ત્યાં જ મારા મૂડની એને ખબર પડે...
        એ મુક્ત વ્યક્તિ છે, સ્વભાવે, પ્રકૃતિએ, વિચારે અને વ્યવહારે પણ. એ બીજાને મુક્ત રાખવામાં માને છે, કારણ કે પોતે બંધાઈ શકે એમ નથી. એણે ક્યારેય મને "વૈદ્ય" ઉમેરવા વિશે, મારા વ્યવસાય વિશે કે આવક વિશે સવાલો કરવાની કે સલાહ આપવાની તસ્દી લીધી નથી. એ સીધા પ્રશ્નો ન પૂછે, એને મારા વિશે જાણવું કે કાળજી કરવી ગમે-પણ એની અનુકૂળતાએ. એ વ્યવહારુ નથી. લેવડ-દેવડ એને નથી આવડતી એમ કહી શકાય. એ બદમિજાજ નથી, ઓછાબોલો છે. બેદરકાર પણ નથી, બેફિકર ચોક્કસ છે. એની બેફિકરાઈ ક્યારેક આપણને ખૂંચે, કારણ કે આપણે એની પાસેથી અપેક્ષા રાખીએ... એ અપેક્ષા ન રાખે એટલે એને નિરાશા ઓછી જ થાય. 
         એકવાર એક પાર્ટીમાં એક બહેન બીજાને કહેતાં હતાં, "આ ભાઈ ઊભા છે ને એ કાજલબેનના 'એક્સ' હસબન્ડ છે." સંજયે એમના ખભે હાથ મૂકીને કહ્યું, "એમ ? તો કાજલે મને કેમ નથી કહ્યું ?" એની સેન્સ ઓફ હ્યુમર ક્યારેક સટલ છે તો ક્યારેક સ્ટુપીડ... બધાને સમજાતી નથી, ને કેટલાકને પચતી યે નથી ! એની સેન્સ ઓફ હ્યુમરનું ટાઈમિંગ ક્યારેક એટલું ખરાબ હોય કે આપણો ગુસ્સો ચાર ગણો વધી જાય, તો ક્યારેક ભયાનક ગુસ્સામાં હોઈએ ત્યારે એની વાત સાંભળીને હસવું આવી જાય. એ ઝઘડતા ઝઘડતા ભૂલી જાય કે અમે કઈ વાતે ઝઘડો શરૂ કર્યો હતો !
         અમે બંને સ્વભાવે અત્યંત મુક્ત, અને સ્વતંત્ર છીએ. ખાસ કરીને, મારો ઉછેર આજથી પચાસ વર્ષ પહેલાં, 'એક દીકરીનો થવો જોઈએ' એવો નથી થયો. મને નિર્ણય કરવાની, ભૂલો કરવાની, અથડાવા-કૂટાવાની, પરિણામ ભોગવવાની પૂર્ણ સ્વતંત્રતા મળી છે. ઘણા પુરુષો જે મારી સાથે વ્યવસાય કે મિત્રતાના સંબંધે સંકળાયા હોય એમને માટે આજે પણ હું "ઝૂમાંથી આવેલું પ્રાણી" છું. એમને હું સમજાતી નથી, સ્વીકારવાનો પ્રશ્ન તો પછી આવે છે ! 
        સંજય મને સમજે છે એટલું તો મારે સ્વીકારવું જ પડે. એણે મને સ્વતંત્રતા આપી છે, અલબત્ત પોતાની સ્વતંત્રતાની શરતે ! અમે બંને અત્યંત સારા મિત્રો છીએ. લડી શકીએ, ઝઘડી શકીએ, ખુલ્લા દિલે એકબીજાનો વિરોધ કરી શકીએ, એકબીજાની મજાક કરી શકીએ અને માતા-પિતા તરીકે પૂરી સમજણ અને કન્સર્નથી એકબીજાને સહકાર આપી શકીએ. અમારી પાસે ત્રણ ઘર છે, "મારું" કે "તારું" એવી વહેંચણીને બદલે અમે એ ઘરોને એડ્રેસથી ઓળખીએ છીએ. ચંદ્રકોલોની, ગાલા અને પલક... અમારા પરિવારના ચાર જણ, જેને જ્યાં ફાવે ત્યાં રહે છે, જ્યારે, જેમ અનુકૂળ હોય તેમ !
        અમે કદાચ આઈડિયલ કપલ નથી, અથવા અમે જ આઈડિયલ કપલ છીએ ! એક છત નીચે રહેવું એ દામ્પત્ય નથી, એક જ વિચાર, એક જ શોખ, એક જ સ્વાદ કે સતત એકબીજાને ગમતા રહેવું એ પણ દામ્પત્ય નથી જ... આવું અમે બંને માનીએ છીએ.
        હું ભાગ્યે જ એના વિશે વાત કરું છું, ને એ મારે વિશે વાત કરતો જ નથી. ઘણા લોકો માને છે કે અમારી વચ્ચે કોઈ વાંધો છે કે પછી અમારા લગ્ન તૂટી ગયાં છે. આજે, અમારા લગ્નને 27 વર્ષ પૂરાં થાય છે ત્યારે આ લખું છું. હું લીગલી, ઓફિસિયલી, સોશિયલી, ઈમોશનલી એની પત્ની છું.
લોકો શું માને છે એની પરવાહ મેં ક્યારેય કરી નથી, મારા સાસુ, મારો દીકરો, કે મારા મિત્રો શું માને છે એટલા પૂરતી મારી જવાબદારી છે અને રહેશે. મારા સસરા જીવ્યા ત્યાં સુધી મને 'કાજલ ગૌરી' કહેતા ! એમણે અમારા બંનેની આ સ્વતંત્રતાને પૂરા સ્નેહ અને સ્વીકારથી સન્માન આપ્યું હતું. 
        મારી મરણનોંધ 'કાજલ ઓઝા વૈદ્ય'ના નામે છપાય તો, અમારે વિશે કૂતુહલ ધરાવનારને નવાઈ ન લાગવી જોઈએ ! """

import codecs
import string
import re
from bs4 import BeautifulSoup  
from nltk.tokenize import word_tokenize
from indicnlp.tokenize import sentence_tokenize
from indicnlp.tokenize import indic_tokenize
import pandas as pd
import sklearn as sk
import math
import io



tokenize_words=[]
originalsen=[]
my_id = 0

   
for tokenize in indic_tokenize.trivial_tokenize(text):
    tokenize_words.append(tokenize)
# Convert list of strings to string
originalfull_str = convert_list_to_string(tokenize_words)
senlateron=[]
sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
for t in sentences:
    senlateron.append(t)
originalsen=[]
sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
for t in sentences:
    sen=removestopwords(t)
    originalsen.append(sen)
    #print(originalsen)
sen_listfortfidf=tokenizefortfidf(originalsen)
#print(sen_listfortfidf)
list3 = [''.join(c for c in s if c not in string.punctuation) for s in sen_listfortfidf]
list3 = [s for s in list3 if s]
dictionary = wordListToFreqDict(list3)
sorteddict = sortFreqDict(dictionary)
context=listToString(list3)
wordDictA = dict.fromkeys(list3, 0)
for word in list3:
    wordDictA[word]+=1
df2=pd.DataFrame([wordDictA])
tfFirst = computeTF(wordDictA, context)
tf = pd.DataFrame([tfFirst])
idfs = computeIDF(wordDictA)
idfFirst = computeTFIDF(tfFirst, idfs)
#idfSecond = computeTFIDF(tfSecond, idfs)
#putting it in a dataframe
idf= pd.DataFrame([idfFirst])
idf = idf.transpose()
idf.columns=['ratio']
final_df = pd.DataFrame(columns=['sentence', 'score'])

for i in range(len(originalsen)):
    originalsen[i]=str(originalsen[i])[1:-1]
    word = listToString(originalsen[i])

    word1 = word.split()

    lenofdf=len(idf)
    loopfirst(lenofdf,word1,idf)
    total = sum(lst)
    final_df = final_df.append({'sentence': originalsen[i], 'score': total},ignore_index='True')

    lst.clear()
final_df.sort_values(by=['score'], inplace=True, ascending=False)
#final_df1=final_df.sort_values(by=['score'], inplace=True, ascending=False)
#print(final_df)
final_df2=final_df['sentence'].head(round(40*len(final_df.index)/100))
#print("this is final_df2",final_df2)
listtoretrieve=list(final_df2.index.values) 
listtoretrieve.sort()
#print(listtoretrieve)
#         #final_df.sort_values(by=['score'], inplace=True, ascending=False)
#from original list retrive
df2 = pd.DataFrame(senlateron,columns=['sentences'])
new_list = df2.values.tolist()
res1=[]

for x in new_list:
    res1=res1+x
finaldraft=""
for i in listtoretrieve:
    finaldraft+=res1[i]
finaldraft=" ".join(finaldraft.split())   
print(finaldraft)
originalfull_str=finaldraft

અમદાવાદના એક જાણીતા અખબારમાં ચાર કોલમનું બોક્સ એક છોકરી વિશે છપાયેલું .એના પિતા સાથેના કોઈક વાંધાને કારણે એના ચારિત્ર્ય ઉપર સાચા - ખોટા છાંટા ઉડાડવામાં આવેલા .એ બદમિજાજ નથી , ઓછાબોલો છે .બેદરકાર પણ નથી , બેફિકર ચોક્કસ છે .એની બેફિકરાઈ ક્યારેક આપણને ખૂંચે , કારણ કે આપણે એની પાસેથી અપેક્ષા રાખીએ . . . એ અપેક્ષા ન રાખે એટલે એને નિરાશા ઓછી જ થાય .એકવાર એક પાર્ટીમાં એક બહેન બીજાને કહેતાં હતાં , " આ ભાઈ ઊભા છે ને એ કાજલબેનના ' એક્સ ' હસબન્ડ છે ." સંજયે એમના ખભે હાથ મૂકીને કહ્યું , " એમ ?તો કાજલે મને કેમ નથી કહ્યું ?" એની સેન્સ ઓફ હ્યુમર ક્યારેક સટલ છે તો ક્યારેક સ્ટુપીડ . . . બધાને સમજાતી નથી , ને કેટલાકને પચતી યે નથી !એની સેન્સ ઓફ હ્યુમરનું ટાઈમિંગ ક્યારેક એટલું ખરાબ હોય કે આપણો ગુસ્સો ચાર ગણો વધી જાય , તો ક્યારેક ભયાનક ગુસ્સામાં હોઈએ ત્યારે એની વાત સાંભળીને હસવું આવી જાય .એ ઝઘડતા ઝઘડતા ભૂલી જાય કે અમે કઈ વાતે ઝઘડો શરૂ કર્યો હતો !અમે બંને સ્વભાવે અત્યંત મુક્ત , અને સ્વતંત્ર છીએ .ખાસ કરીને , મારો ઉછેર આજથી પચાસ વર્ષ પહેલાં , ' એક દીકરીનો થવો જોઈએ ' એવો નથી થયો .મને નિર્ણય કરવાની , ભૂલો કરવાન

In [28]:
tokenize_words1=[]
for tokenize in indic_tokenize.trivial_tokenize(originalfull_str):
    tokenize_words.append(tokenize)
        # Convert list of strings to string
originalfull_str = convert_list_to_string(tokenize_words)
senlateron=[]
sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
i=1
# data1="""વાઈવા આપતી વખતે, એપ્રનના પહેલા બે બટન ખોલી નાંખતી હશે.એમબીબીએસમાં મારી જ સાથે ભણતા એક મેડિકલ સ્ટુડન્ટે અમારા ક્લાસની ટોપર અને સૌથી હોંશિયાર છોકરી વિશે કરેલી ટિપ્પણી મને આજે પણ યાદ છે.
#     આપણી આસપાસ એવા અઢળક લોકો છે જે એવું માને છે કે સફળતાનો રસ્તો પુરુષ કરતા સ્ત્રી માટે વધારે ટૂંકો હોય છે.
#     સફળતાની સીડીઓ ચડી રહેલી સ્ત્રીને પાછળ ધકેલવા અને એણે મેળવેલી સિદ્ધિઓનો અનાદર કરવા માટે અન્ય કોઈ સ્ત્રી કે પુરુષ પાસે રહેલું  સૌથી  જુનું
#     અને ધારદાર  હથિયાર એટલે  'કેરેક્ટર એસેસીનેશન.'
#     ફેમિનિઝમના સહારે અને ભરોસે રહેવાને બદલે એક સ્ત્રી જ્યારે પોતાના હકની સિદ્ધિ અને સફળતા, પોતાની મહેનત અને ટેલેન્ટના
#     જોર પર ઝૂંટવી લે છે ત્યારે વાત એના ચારિત્ર્ય પર આવે છે.
#     ઈસ્ટ્રોજન સામેની દોડમાં જ્યારે ટેસ્ટોસ્ટેરોન ઘવાય છે, ત્યારે વાત એના ચારિત્ર્ય પર આવે છે. જે સમયે કોઈપણ જાતના જજમેન્ટ
#     પાસ કર્યા વગર એક આગળ વધી રહેલી સ્ત્રીની સફળતાને આપણે 'એઝ ઈટ ઈઝ' સ્વીકારતા થઈ જશું, ત્યારે આપણે સિવિલાઈઝ્ડ 
#     કહેવાશું. ખ્યાતિ, લોકપ્રિયતા કે પ્રસિદ્ધિ મેળવવા માટે દરેક સ્ત્રીએ 'કોમ્પ્રોમાઈઝ' કરવું જ પડે છે,
#     એવું માનનારા લોકો પોતાની સમજણ સાથે કોમ્પ્રોમાઈઝ કરી રહ્યા છે."""

# data1="""All said and done, મને મળી રહેલા પર્સનલ મેસેજીસની સંખ્યા 'અનમેનેજબલ' થઈ રહી છે.મેઈલ કે મેસેજ દ્વારા જે રીતે અને
# જે સંખ્યામાં લોકો પોતાની ઉદાસી કબુલી રહ્યા છે, પોતાને બચાવવા માટે રીક્વેસ્ટ કરી રહ્યા છે.
#     ૧.સૌથી પહેલા તાત્કાલિક કોઈ થેરાપીસ્ટ, મનોચિકિત્સક કે કાઉન્સેલરને મળો.સારવારનો પહેલો તબક્કો સ્વીકાર છે.
#     ડિપ્રેશનની દવા અસર કરતા ૧૫-૨૦ દિવસ લાગે છે.
#     ત્યાં સુધી 'નો સ્યુસાઈડ કોન્ટ્રેક્ટ' સાઈન કરો.
# ૨.'મેન્ટલ ડાયેરીયા' બંધ કરવા માટે મગજને બિનજરૂરી અને વાહિયાત માહિતીઓ પીરસવાનું બંધ કરો.
# એન્ડ ઈન પ્રોસેસ ઓફ ડૂઇંગ સો, સૌથી પહેલા સોશિયલ મીડિયા અને ન્યુઝ ચેનલ્સ બંધ કરો.
# ફેસબુક, ઇન્સ્ટા, વ્હોટ્સ-એપ પરથી થોડો સમય બ્રેક લઈ લો.
# સોશિયલ મીડિયા એ ડિપ્રેશનનું સૌથી મોટું કારણ છે કારણકે Every time we sign into it, unknowingly we are landing into the zone
# of comparison.સંપર્ક માટેના ત્રણ શ્રેષ્ઠ રસ્તાઓ જ ખુલ્લા રાખો.
#     'રૂટીન પ્રાયોરીટી' કામ હોય તો ઈ-મેઈલ, સેમી-અરજન્ટ હોય તો ટેક્સ્ટ મેસેજ (SMS) અને બહુ અરજન્ટ હોય તો ફોન કોલ.
#     બ્રેક ન લો તો પણ, રીસર્ચ એવું કહે છે કે સોશિયલ મીડિયા પર ૩૦ મીનીટ કે ૧ કલાકથી વધારે સમય ન પસાર કરો.
# ૩.We all are constantly surrounded by people full of shit, and only Meditation can rescue us.
#     'ગાઇડેડ મેડિટેશન' માટે તમારા ફોનમાં 'Calm' કે 'Headspace' એપ ડાઉનલોડ કરો.
# 'https://www.thinklessandgrowrich.com/meditation-for-beginners-the-complete-guide/'.
# ૪.પુસ્તકો વાચો.Your brain deserves much better content than non-sense forwards, silly jokes, and good morning messages.
# Few good reads are : 1.Kick the thinking habit by Richard Paterson
# 2.The subtle art of not giving a F*ck  by Mark Manson
# 3.The power of Now by Eckhart Tolle 
# 4.Any book by Dalai Lama
# 5.Any book by Mahatma Gandhi
# 6.Any book on Buddha and Buddhism
# 7.વિનોબાજીના ગીતા પ્રવચનો
# ૫.Nature heals the soul.ક્યારેય ડિપ્રેશન નહીં આવે. હવે, તો આખી ફોરેસ્ટ મેડિસિનની બ્રાંચ ડેવલપ થઈ રહી છે, જે કહે છે કે 'If you hug a tree,
# It will make you happy.'
# ૬. મનમાં આવતા નેગેટીવ, નિરાશાજનક અને વાહિયાત વિચારોનું ઉત્સર્જન કરો.
# ડિપ્રેશન એ બીજું કાંઈ નથી પણ અવ્યક્ત રહી ગયેલા એટેન્શન સીકિંગ વિચારોનો આપણી બુદ્ધિ સામે થયેલો બળવો છે.
# ૭.ગ્રેટીટ્યુડ ટેકનીક : રોજ સવારે ઉઠીને સૌથી પહેલા એક કાગળ પર એવી ત્રણ વસ્તુઓ લખો, જેના માટે તમે ઈશ્વરના આભારી છો.
# એ કાંઈ પણ હોય શકે છે.
# આવું બધું વિચારશું, તો મરવાની ઈચ્છા નહીં થાય.
# ૮.એક્સરસાઈઝનો સૌથી મોટો ફાયદો માનસિક સ્વાસ્થ્ય પર થાય છે.
# જીમ કરો, સાયકલીંગ, રનીંગ, સ્કીપીંગ, સ્વીમીંગ કાંઈ પણ કરો.
# કાંઈ નહીં તો ગો ઓન અ લોંગ વોક.
# ૯.તમારા હેડ-ફોન્સ લગાડો અને તમારા ગમતા પ્લે-લીસ્ટમાં ખોવાઈ જાવ.
# ૧૦.ધ લીસ્ટ ઈઝ એન્ડ-લેસ. હજી ઘણું બાકી છે."""

data1=""" અમદાવાદના એક જાણીતા અખબારમાં ચાર કોલમનું બોક્સ એક છોકરી વિશે છપાયેલું.
    એના પિતા સાથેના કોઈક વાંધાને કારણે એના ચારિત્ર્ય ઉપર સાચા-ખોટા છાંટા ઉડાડવામાં આવેલા.
     આજથી અઢી દાયકા પહેલાં ! સવારના દસ વાગ્યા પહેલાં નહીં ઉઠનારો એનો બોયફ્રેન્ડ સાડા આઠ વાગ્યામાં ખમણ અને ગાંઠિયા લઈને એના ઘરે પહોંચ્યો.
     સહુએ સાથે નાસ્તો કર્યો એટલું જ નહીં, એણે હસીને કહ્યું, "લખ્યા કરે... મારે માટે તું વધારે સાચી કે આ લખનાર ?"
આજથી અઢી દાયકા પહેલાં, એકવાર ડીવોર્સ થઈ ચૂક્યા હોય એવી છોકરી સાથે લગ્ન કરનારો કુંવારો છોકરો એને એકવાર પણ પૂછે નહીં, "એક્ચ્યુઅલી શું થયેલું ?" તો એને માટે સન્માન થયા વગર રહે ? જોકે,
 એક વર્ષથી આપણે સાથે ફરીએ છીએ.
    હવે આપણે પરણી જવું જોઈએ એવું તને નથી લાગતું જો હું ના પાડું, તો ટ્રેનના અવાજમાં એ ના દબાઈ જાય.
    સાથે જીવવાની ઈચ્છા હતી,કારણ એ, કે મારા પ્રથમ લગ્નમાં હું અપમાન, દાદાગીરી અને ડોમેસ્ટીક.
    આજે પણ !નામ એનું સંજય વૈદ્ય.
    વ્યવસાયે ફોટોગ્રાફર, ને સ્વભાવે અલગારી.
    ત્યારે ગુજરાત સમાચારમાં નોકરી કરતો,
 એ સીધા પ્રશ્નો ન પૂછેએ વ્યવહારુ નથી.
    લેવડ-દેવડ એને નથી આવડતી એમ કહી શકાય.
    એ બદમિજાજ નથી, ઓછાબોલો છે.
    એની બેફિકરાઈ ક્યારેક આપણને ખૂંચે, કારણ કે આપણે એની પાસેથી અપેક્ષા રાખીએ.
    એ અપેક્ષા ન રાખે એટલે એને નિરાશા ઓછી જ થાય.
   એની સેન્સ ઓફ હ્યુમર ક્યારેક સટલ છે તો ક્યારેક સ્ટુપીડ!.
અમે બંને સ્વભાવે અત્યંત મુક્ત, અને સ્વતંત્ર છીએ.
    ખાસ કરીને, મારો ઉછેર આજથી પચાસ વર્ષ પહેલાં, "એક દીકરીનો થવો જોઈએ" એવો નથી થયો.
    ઘણા પુરુષો જે મારી સાથે વ્યવસાય કે મિત્રતાના સંબંધે સંકળાયા હોય એમને માટે આજે પણ હું "ઝૂમાંથી આવેલું પ્રાણી" છું.
    એમને હું સમજાતી નથી, સ્વીકારવાનો પ્રશ્ન તો પછી આવે છે ! સંજય મને સમજે છે એટલું તો મારે સ્વીકારવું જ પડે.
    અમે બંને અત્યંત સારા મિત્રો છીએ.
    અમારી પાસે ત્રણ ઘર છે, "મારું" કે "તારું" એવી વહેંચણીને બદલે અમે એ ઘરોને એડ્રેસથી ઓળખીએ છીએ.
    અમારા પરિવારના ચાર જણ, જેને જ્યાં ફાવે ત્યાં રહે છે,
    આજે, અમારા લગ્નને 27 વર્ષ પૂરાં થાય છે ત્યારે આ લખું છું.
    મારા સસરા જીવ્યા ત્યાં સુધી મને "કાજલ ગૌરી" કહેતા ! એમણે અમારા બંનેની આ સ્વતંત્રતાને પૂરા સ્નેહ અને સ્વીકારથી સન્માન આપ્યું હતું.
    મારી મરણનોંધ "કાજલ ઓઝા વૈદ્ય"ના નામે છપાય તો, અમારે વિશે કૂતુહલ ધરાવનારને નવાઈ ન લાગવી જોઈએ !"""
i=i+1
for tokenize1 in indic_tokenize.trivial_tokenize(data1):
        tokenize_words1.append(tokenize1)
        # Convert list of strings to string
originalfull_str1 = convert_list_to_string(tokenize_words1)
senlateron1=[]
sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
_, _, rouge_1 = rouge_n_summary_level(tokenize_words, tokenize_words1, 1)
#rouge1.append(rouge_1)
print('ROUGE-1: %f' % rouge_1)
_, _, rouge_2 = rouge_n_summary_level(tokenize_words, tokenize_words1, 2)
print('ROUGE-2: %f' % rouge_2)
#rouge2.append(rouge_2)
_, _, rouge_l = rouge_l_summary_level(tokenize_words, tokenize_words1)
print('ROUGE-L: %f' % rouge_l)
#rougel.append(rouge_l)
_, _, rouge_w = rouge_w_summary_level(tokenize_words, tokenize_words1)
print('ROUGE-W: %f' % rouge_w)

ROUGE-1: 0.453171
ROUGE-2: 0.445246
ROUGE-L: 0.453171
ROUGE-W: 0.157529


In [29]:
tokenize_words1=[]
for tokenize in indic_tokenize.trivial_tokenize(originalfull_str):
    tokenize_words.append(tokenize)
        # Convert list of strings to string
originalfull_str = convert_list_to_string(tokenize_words)
senlateron=[]
sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
i=1
# data1="""વાઈવા આપતી વખતે, એપ્રનના પહેલા બે બટન ખોલી નાંખતી હશે.એમબીબીએસમાં મારી જ સાથે ભણતા એક મેડિકલ સ્ટુડન્ટે અમારા 
#       ક્લાસની ટોપર અને સૌથી હોંશિયાર છોકરી વિશે કરેલી ટિપ્પણી મને આજે પણ યાદ છે.પુરુષની સફળતાને એના સાહસ,
#       સમર્પણ અને મહેનત સાથે જોડવામાં આવે છે  જ્યારે  સ્ત્રીની સફળતાને શક્યતાઓ સાથે. એ કોઈ કલાકાર હોય, જર્નલીસ્ટકે 
#      ઓફિસ વર્કર. કોઈપણ ક્ષેત્રમાં એક સ્ત્રી જ્યારે અચાનક લાઈમ-લાઈટમાં આવે છે, ત્યારે એની પ્રસિદ્ધિનો યશ એની પ્રતિભા કરતા એની સુંદરતા, એના 
#      ફિઝીક અને એની 'ઉદારતા' ને  આપવામાં આવે છે.કારણકે ફેમિનિઝમ તો સ્ત્રીઓને સમાન હક અને તક અપાવવા માટે હોય છે.
#      ઈસ્ટ્રોજન સામેની દોડમાં જ્યારે ટેસ્ટોસ્ટેરોન ઘવાય છે, ત્યારે વાત એના ચારિત્ર્ય પર આવે છે.સ્ત્રીના સક્સેસને એની સેક્સ્યુઆલીટી સાથે જોડી દેવામાં આવે છે.
# સ્ત્રીની સફળતા એણે કરેલું સમાધાન નથી, એણે કરેલી સાધના છે.કુટુંબને આપેલા સમય સાથે, પોતાના મોજ-શોખ અને સુખ-સગવડો સાથે.
# એક સ્ત્રી જ્યારે પોતાના 'કમ્ફોર્ટ ઝોન' માંથી બહાર નીકળે છે, ત્યારે એની સફળતા બીજા અનેક લોકો માટે 'અનકમ્ફોર્ટેબલ' બની જતી હોય છે."""
# data1="""All said and done, મને મળી રહેલા પર્સનલ મેસેજીસની સંખ્યા 'અનમેનેજબલ' થઈ રહી છે.
# મેઈલ કે મેસેજ દ્વારા જે રીતે અને જે સંખ્યામાં લોકો પોતાની ઉદાસી કબુલી રહ્યા છે.
# એ જોઈને હું સ્તબ્ધ છું.So I thought, Let me share something from my personal experience.
# ૧.સૌથી પહેલા તાત્કાલિક કોઈ થેરાપીસ્ટ, મનોચિકિત્સક કે કાઉન્સેલરને મળો.સારવારનો પહેલો તબક્કો સ્વીકાર છે.ડિપ્રેશનની દવા અસર કરતા ૧૫-૨૦ દિવસ લાગે છે.
# ત્યાં સુધી 'નો સ્યુસાઈડ કોન્ટ્રેક્ટ' સાઈન કરો. 
# ૨.'મેન્ટલ ડાયેરીયા' બંધ કરવા માટે મગજને બિનજરૂરી અને વાહિયાત માહિતીઓ પીરસવાનું બંધ કરો.એન્ડ ઈન પ્રોસેસ ઓફ ડૂઇંગ સો, સૌથી પહેલા
# સોશિયલ મીડિયા અને ન્યુઝ ચેનલ્સ બંધ કરો.સોશિયલ મીડિયા એ ડિપ્રેશનનું સૌથી મોટું કારણ.
# ૩.મેડિટેશન.મેડિટેશન.મેડિટેશન.આજ નહીં તો કાલ નહીં તો વર્ષો પછી, મેડિટેશન સિવાય ઉદ્ધાર નથી.તમારે કોઈ શીબીર કે ધ્યાન ગુરુ પાસે જવાની જરૂર નથી.
# એ તમે ઘેર બેઠા કરી શકો છો અને એ પણ એકલા.
# 'ગાઇડેડ મેડિટેશન' માટે તમારા ફોનમાં Calm કે Headspace એપ ડાઉનલોડ કરો.
# હું હેડસ્પેસ યુઝ કરું છું એન્ડ આઈ એમ લવીંગ ઇટ.
# ૪.પુસ્તકો વાચો. Few good reads are : 
# 1.Kick the thinking habit by Richard Paterson 
# 2.The subtle art of not giving a F*ck  by Mark Manson 
# 3.The power of Now by Eckhart Tolle 
# 4.Any book by Dalai Lama
# 5. Any book by Mahatma Gandhi
# 6.Any book on Buddha and Buddhism
# 7.'વિનોબાજીના ગીતા પ્રવચનો' 

# ૫.ઈકો-થેરાપી : Nature heals the soul.
# 'If you hug a tree, It will make you happy.'  
# ૬.મનમાં આવતા નેગેટીવ, નિરાશાજનક અને વાહિયાત વિચારોનું ઉત્સર્જન કરો.
# ડિપ્રેશન એ બીજું કાંઈ નથી પણ અવ્યક્ત રહી ગયેલા એટેન્શન સીકિંગ વિચારોનો આપણી બુદ્ધિ સામે થયેલો બળવો છે.
# કેથાર્સીસ કે વિરેચન માટેનું કોઈ એક માધ્યમ શોધી કાઢો.
# ૭.ગ્રેટીટ્યુડ ટેકનીક : રોજ સવારે ઉઠીને સૌથી પહેલા એક કાગળ પર એવી ત્રણ વસ્તુઓ લખો, જેના માટે તમે ઈશ્વરના આભારી છો.
# એ કાંઈ પણ હોય શકે છે.જેમકે તમે જીવો છો. યુનિસેફના આંકડા પ્રમાણે વિશ્વના ૨૨,૦૦૦ બાળકો દરરોજ ભૂખમરાથી મૃત્યુ પામે છે.
# આવું બધું વિચારશું, તો મરવાની ઈચ્છા નહીં થાય.
# ૮.ભલે આળસ આવે, પણ પરાણે કસરત કરો.
# કાંઈ નહીં તો ગો ઓન અ લોંગ વોક.ધિસ વિલ મેક યુ ફીલ ગુડ.
# ૯.તમારા હેડ-ફોન્સ લગાડો અને તમારા ગમતા પ્લે-લીસ્ટમાં ખોવાઈ જાવ.
# ૧૦. ધ લીસ્ટ ઈઝ એન્ડ-લેસ.હજી ઘણું બાકી છે."""
data1="""અમદાવાદના એક જાણીતા અખબારમાં ચાર કોલમનું બોક્સ એક છોકરી વિશે છપાયેલું.
એના પિતા સાથેના કોઈક વાંધાને કારણે એના ચારિત્ર્ય ઉપર સાચા-ખોટા છાંટા ઉડાડવામાં આવેલા.
"છાપામાં છપાઈ ગયું..." એ કેટલી ભયાનક વાત હોઈ શકે, આજથી અઢી દાયકા પહેલાં !
સવારના દસ વાગ્યા પહેલાં નહીં ઉઠનારો એનો બોયફ્રેન્ડ સાડા આઠ વાગ્યામાં ખમણ અને ગાંઠિયા લઈને એના ઘરે પહોંચ્યો... સહુએ સાથે નાસ્તો કર્યો એટલું જ નહીં,
એણે હસીને કહ્યું, "લખ્યા કરે... મારે માટે તું વધારે સાચી કે આ લખનાર ?"
એકવાર ડીવોર્સ થઈ ચૂક્યા હોય એવી છોકરી સાથે લગ્ન કરનારો કુંવારો છોકરો એને એકવાર પણ પૂછે નહીં, "એક્ચ્યુઅલી શું થયેલું ?" 
"એક વર્ષથી આપણે સાથે ફરીએ છીએ.
મારા મા-બાપ રોજ પૂછે છે... હવે આપણે પરણી જવું જોઈએ એવું તને નથી લાગતું ?"  
જો હું ના પાડું, તો ટ્રેનના અવાજમાં એ ના દબાઈ જાય... લગ્ન વિશે મારા મનમાં બહુ મોટો અવઢવ હતો, 
કારણ એ, કે મારા પ્રથમ લગ્નમાં હું અપમાન, દાદાગીરી અને ડોમેસ્ટીક વાયોલેન્સનો ભોગ બની ચૂકી હતી. 
એના મનમાં ક્લેરીટી હતી.
પરણવું છે !
એ મુક્ત વ્યક્તિ છે, સ્વભાવે, પ્રકૃતિએ, વિચારે અને વ્યવહારે પણ.
એણે ક્યારેય મને "વૈદ્ય" ઉમેરવા વિશે, મારા વ્યવસાય વિશે કે આવક વિશે સવાલો કરવાની કે સલાહ આપવાની તસ્દી લીધી નથી.
લેવડ-દેવડ એને નથી આવડતી એમ કહી શકાય.
એની બેફિકરાઈ ક્યારેક આપણને ખૂંચે,કારણ કે આપણે એની પાસેથી અપેક્ષા રાખીએ... એ અપેક્ષા ન રાખે એટલે એને નિરાશા ઓછી જ થાય.
એની સેન્સ ઓફ હ્યુમરનું ટાઈમિંગ ક્યારેક એટલું ખરાબ હોય કે આપણો ગુસ્સો ચાર ગણો વધી જાય, તો ક્યારેક ભયાનક ગુસ્સામાં હોઈએ ત્યારે એની વાત 
સાંભળીને હસવું આવી જાય.
અમે બંને સ્વભાવે અત્યંત મુક્ત, અને સ્વતંત્ર છીએ.
મારો ઉછેર આજથી પચાસ વર્ષ પહેલાં, "એક દીકરીનો થવો જોઈએ" એવો નથી થયો.
ઘણા પુરુષો જે મારી સાથે વ્યવસાય કે મિત્રતાના સંબંધે સંકળાયા હોય એમને માટે આજે પણ હું "ઝૂમાંથી આવેલું પ્રાણી" છું.
અમે બંને અત્યંત સારા મિત્રો છીએ.
અમારી પાસે ત્રણ ઘર છે.
આજે, અમારા લગ્નને 27 વર્ષ પૂરાં થાય છે ત્યારે આ લખું છું.
મારા સાસુ, મારો દીકરો, કે મારા મિત્રો શું માને છે એટલા પૂરતી મારી જવાબદારી છે અને રહેશે."""
i=i+1
for tokenize1 in indic_tokenize.trivial_tokenize(data1):
        tokenize_words1.append(tokenize1)
        # Convert list of strings to string
originalfull_str1 = convert_list_to_string(tokenize_words1)
senlateron1=[]
sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
_, _, rouge_1 = rouge_n_summary_level(tokenize_words, tokenize_words1, 1)
#rouge1.append(rouge_1)
print('ROUGE-1: %f' % rouge_1)
_, _, rouge_2 = rouge_n_summary_level(tokenize_words, tokenize_words1, 2)
print('ROUGE-2: %f' % rouge_2)
#rouge2.append(rouge_2)
_, _, rouge_l = rouge_l_summary_level(tokenize_words, tokenize_words1)
print('ROUGE-L: %f' % rouge_l)
#rougel.append(rouge_l)
_, _, rouge_w = rouge_w_summary_level(tokenize_words, tokenize_words1)
print('ROUGE-W: %f' % rouge_w)

ROUGE-1: 0.239392
ROUGE-2: 0.236421
ROUGE-L: 0.239392
ROUGE-W: 0.087268
